In [1]:
import numpy as np
import random
import Simulated_annealing
from heapdict import heapdict

class Move_Costs(np.ndarray):

    def __new__(cls, input_array):
        # Cast the existing array to be an instance of this class
        obj = np.asarray(input_array).view(cls)
        return obj
    
    def popitem(self):
        row, col = np.unravel_index(np.argmin(self), shape = self.shape)
        return (row, col), self[row][col]
        

class TSP(Simulated_annealing.State):
    def __init__(self, graph : np.ndarray):
        self.graph = graph
        self.n_nodes = graph.shape[0]

        unexplored = list(range(self.n_nodes))
        curr = 0

        self.state = []
        self.state.append(curr)
        unexplored.remove(curr)

        for i in range(1, self.n_nodes):
            curr = unexplored[np.argmin(self.graph[curr, unexplored])]
            self.state.append(curr)
            unexplored.remove(curr)
        
        #self.history = [self.state]

        
    def get_neighbour(self):
        i = random.randrange(0, self.n_nodes)
        j = i
        
        while (j == i):
            j = random.randrange(0, self.n_nodes)
        
        i, j = min(i, j), max(i, j)

        while (i == 0 and j == self.n_nodes - 1):
            i = random.randrange(0, self.n_nodes)
            j = i
            while (j == i):
                j = random.randrange(0, self.n_nodes)
            
            i, j = min(i, j), max(i, j)

        return (i, j)
    
    def cost(self, state):
        if state is None:
            state = self.state

        cost = 0
        for i in range(0, self.n_nodes):
            cost += self.graph[state[i], state[(i+1) % self.n_nodes]]
        
        return cost
    
    def cost_change(self, move):
        i, j = move
        i, j = min(i, j), max(i, j)

        if (i == 0 and j == self.n_nodes - 1) or (i == self.n_nodes - 1 and j == 0):
            return 0
        
        i_prev_node = self.state[i-1] 
        i_node = self.state[i]
        j_node = self.state[j]
        j_next_node = self.state[(j + 1) % self.n_nodes] 

        old_cost = self.graph[i_prev_node, i_node] + self.graph[j_node, j_next_node]
        new_cost = self.graph[i_prev_node, j_node] + self.graph[i_node, j_next_node]

        return new_cost - old_cost

    def update(self, move):
        i, j = move
        i, j = min(i, j), max(i, j)

        self.state = self.state[0 : i] + list(reversed(self.state[i : j+1])) + self.state[j+1:]

        #self.history.append(self.state)
    
    def get_all_neighbours(self):
        n = self.n_nodes
        state_arr = np.array(self.state)

        move_matrix = np.full((n, n), np.inf)

        # Get indices for Upper Triangle
        U, V = np.triu_indices(n, k=1)
        
        # Vectorized Cost Calculation
        prev_u_indices = (U - 1) % n
        curr_u_indices = U
        curr_v_indices = V
        next_v_indices = (V + 1) % n

        node_prev_u = state_arr[prev_u_indices]
        node_curr_u = state_arr[curr_u_indices]
        node_curr_v = state_arr[curr_v_indices]
        node_next_v = state_arr[next_v_indices]

        old_costs = self.graph[node_prev_u, node_curr_u] + self.graph[node_curr_v, node_next_v]
        new_costs = self.graph[node_prev_u, node_curr_v] + self.graph[node_curr_u, node_next_v]
        
        deltas = (new_costs - old_costs).astype(float)
        
        # Edge case: wrap-around move (0, N-1) is usually invalid/0
        mask_edge_case = ((U == 0) & (V == n - 1))
        deltas[mask_edge_case] = np.inf 

        # Fill Matrix
        move_matrix[U, V] = deltas
        move_matrix = Move_Costs(move_matrix)
        
        return move_matrix
    
    
    """def get_affected_moves(self, move, queue):
        i, j = move
        i, j = min(i, j), max(i, j)

        initial_state_backup = self.state[:]
        self.update(move)
        
        n = self.n_nodes
        
        # Create a boolean mask of the affected area
        mask = np.zeros((n, n), dtype=bool)
        
        mask[i:j+1, :] = True
        mask[:, i:j+1] = True
        
        # Only keep the Upper Triangle (u < v)
        mask = np.triu(mask, k=1)
        U, V = np.nonzero(mask)

        #Vectorized Cost Logic (Same as get_all_neighbours)
        state_arr = np.array(self.state)
        
        prev_u_indices = (U - 1) % n
        curr_u_indices = U
        curr_v_indices = V
        next_v_indices = (V + 1) % n
        
        node_prev_u = state_arr[prev_u_indices]
        node_curr_u = state_arr[curr_u_indices]
        node_curr_v = state_arr[curr_v_indices]
        node_next_v = state_arr[next_v_indices]
        
        old_costs = self.graph[node_prev_u, node_curr_u] + self.graph[node_curr_v, node_next_v]
        new_costs = self.graph[node_prev_u, node_curr_v] + self.graph[node_curr_u, node_next_v]
        
        deltas = (new_costs - old_costs).astype(float)
        
        mask_edge_case = ((U == 0) & (V == n - 1))
        deltas[mask_edge_case] = np.inf

        queue[U, V] = deltas

        # Revert state
        self.state = initial_state_backup

        return queue"""

In [2]:
%matplotlib notebook

In [3]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML # Required for animation in Jupyter

plt.rcParams['animation.embed_limit'] = 50

# --- 1. Graph Generation Function ---

def generate_random_graph(n_nodes, max_distance=10):
    """
    Generates a random, symmetric adjacency matrix for a complete graph.

    Args:
        n_nodes (int): The number of nodes (cities).
        max_distance (int): The maximum distance between any two nodes.

    Returns:
        np.ndarray: The symmetric adjacency matrix (distance matrix).
    """
    if n_nodes < 2:
        return np.array([[0]])

    # Generate a random upper triangle (including the diagonal)
    # The distances are integers for simplicity
    np.random.seed(42) # Optional: set a seed for reproducibility
    upper_triangular = np.random.randint(1, max_distance + 1, size=(n_nodes, n_nodes))

    # Make it symmetric (A[i, j] = A[j, i])
    graph = np.triu(upper_triangular) + np.tril(upper_triangular.T, k=-1)

    # Set the diagonal (distance from a city to itself) to 0
    np.fill_diagonal(graph, 0)

    # Ensure all distances are non-negative
    graph = np.abs(graph)

    return graph


In [19]:
import math


def wavy_schedule(temp, step, const):
    a = 100
    b = 1 - 0.0001
    y = const * (math.cos(a * math.pi * step)**2 * math.exp(math.log(b) * step))
    return y

In [17]:
def wavy_schedule(temp, step, const):
    return temp * (1 - const)

In [9]:
N_NODES = 1000
graph = generate_random_graph(N_NODES, 100)

In [23]:

TSP_State = TSP(graph)
Solver = Simulated_annealing.Annealer(TSP_State, initial_temp = 1000, temperature_schedule = wavy_schedule, scheduling_constant = 1000)
Solver.anneal(unchanged_threshold = 10000, reset_temp = 0.5, n_runs = 20, local_search = True)

print("Completed Annealing")

step_size = 100
#sampled_history = TSP_State.history[::step_size] + [TSP_State.history[-1], TSP_State.state]

#ani = visualize_path_animation(sampled_history, graph)
plt.show()

Run 1 / 20
Temperature : 1000.0000  Best cost : 1418

Run 2 / 20
Temperature : 500.0000  Best cost : 1418

Run 3 / 20
Temperature : 250.0000  Best cost : 1418

Run 4 / 20
Temperature : 125.0000  Best cost : 1418

Run 5 / 20
Temperature : 62.5000  Best cost : 1418

Run 6 / 20
Temperature : 31.2500  Best cost : 1418

Run 7 / 20
Temperature : 15.6250  Best cost : 1418

Run 8 / 20
Temperature : 7.8125  Best cost : 1418

Run 9 / 20
Temperature : 3.9062  Best cost : 1418

Run 10 / 20
Temperature : 1.9531  Best cost : 1418

Run 11 / 20
Temperature : 0.9766  Best cost : 1418

Run 12 / 20
Temperature : 0.4883  Best cost : 1418

Run 13 / 20
Temperature : 0.2441  Best cost : 1418

Run 14 / 20
Temperature : 0.1221  Best cost : 1418

Run 15 / 20
Temperature : 0.0610  Best cost : 1418

Run 16 / 20
Temperature : 0.0305  Best cost : 1418

Run 17 / 20
Temperature : 0.0153  Best cost : 1418

Run 18 / 20
Temperature : 0.0076  Best cost : 1418

Run 19 / 20
Temperature : 0.0038  Best cost : 1418

Run 20 / 

In [12]:
print(TSP_State.cost(None))
print(TSP_State.state)

1144
[904, 720, 897, 692, 865, 750, 988, 998, 851, 745, 732, 921, 833, 714, 760, 700, 935, 802, 741, 831, 585, 731, 512, 633, 624, 677, 850, 688, 669, 740, 631, 928, 628, 433, 830, 634, 680, 721, 748, 844, 115, 471, 228, 141, 295, 622, 129, 405, 467, 316, 237, 496, 367, 383, 300, 662, 530, 914, 229, 247, 350, 523, 250, 334, 232, 856, 927, 985, 89, 75, 14, 990, 590, 756, 540, 543, 569, 816, 681, 514, 647, 330, 167, 252, 204, 223, 389, 106, 127, 258, 98, 984, 431, 267, 520, 410, 264, 292, 347, 562, 184, 580, 937, 41, 82, 25, 758, 586, 734, 579, 854, 806, 655, 704, 743, 729, 742, 910, 828, 752, 650, 762, 822, 812, 581, 683, 869, 466, 217, 568, 251, 344, 382, 727, 533, 321, 444, 220, 360, 575, 957, 886, 976, 873, 900, 953, 738, 723, 577, 509, 566, 541, 620, 505, 345, 475, 635, 153, 423, 504, 600, 712, 409, 454, 791, 287, 123, 336, 289, 92, 603, 384, 427, 386, 435, 595, 259, 492, 458, 539, 434, 441, 302, 403, 352, 640, 470, 519, 254, 637, 440, 420, 438, 871, 797, 823, 763, 675, 767, 698, 47